# ANOVA  - Lab

## Introduction

In this lab, you'll get some brief practice generating an ANOVA table (AOV) and interpreting its output. You'll also perform some investigations to compare the method to the t-tests you previously employed to conduct hypothesis testing.

## Objectives

In this lab you will: 

- Use ANOVA for testing multiple pairwise comparisons 
- Interpret results of an ANOVA and compare them to a t-test

## Load the data

Start by loading in the data stored in the file `'ToothGrowth.csv'`: 

In [2]:
# Your code here
import pandas as pd

# Load the data
df = pd.read_csv('ToothGrowth.csv')

# Display the first few rows to understand the structure
print(df.head())

    len supp  dose
0   4.2   VC   0.5
1  11.5   VC   0.5
2   7.3   VC   0.5
3   5.8   VC   0.5
4   6.4   VC   0.5


## Generate the ANOVA table

Now generate an ANOVA table in order to analyze the influence of the medication and dosage:  

In [3]:
import statsmodels.api as sm
from statsmodels.formula.api import ols


In [4]:
# Your code here
model = ols('len ~ C(supp) + C(dose)', data=df).fit()
anova_table = sm.stats.anova_lm(model, typ=2)

## Interpret the output

Make a brief comment regarding the statistics and the effect of supplement and dosage on tooth length: 

In [5]:
# Your comment here
print(anova_table)

               sum_sq    df          F        PR(>F)
C(supp)    205.350000   1.0  14.016638  4.292793e-04
C(dose)   2426.434333   2.0  82.810935  1.871163e-17
Residual   820.425000  56.0        NaN           NaN


## Compare to t-tests

Now that you've had a chance to generate an ANOVA table, its interesting to compare the results to those from the t-tests you were working with earlier. With that, start by breaking the data into two samples: those given the OJ supplement, and those given the VC supplement. Afterward, you'll conduct a t-test to compare the tooth length of these two different samples: 

In [6]:
# Your code here
from scipy.stats import ttest_ind

# Separate data into OJ and VC groups
oj_data = df[df['supp'] == 'OJ']['len']
vc_data = df[df['supp'] == 'VC']['len']

Now run a t-test between these two groups and print the associated two-sided p-value: 

In [7]:
# Calculate the 2-sided p-value for a t-test comparing the two supplement groups
t_statistic, p_value = ttest_ind(oj_data, vc_data)
print(f"Two-sided p-value for t-test comparing OJ and VC groups: {p_value}")

Two-sided p-value for t-test comparing OJ and VC groups: 0.06039337122412849


## A 2-Category ANOVA F-test is equivalent to a 2-tailed t-test!

Now, recalculate an ANOVA F-test with only the supplement variable. An ANOVA F-test between two categories is the same as performing a 2-tailed t-test! So, the p-value in the table should be identical to your calculation above.

> Note: there may be a small fractional difference (>0.001) between the two values due to a rounding error between implementations. 

In [8]:
# Your code here; conduct an ANOVA F-test of the oj and vc supplement groups.
# Compare the p-value to that of the t-test above. 
# They should match (there may be a tiny fractional difference due to rounding errors in varying implementations)

model_supp = ols('len ~ C(supp)', data=df).fit()
anova_table_supp = sm.stats.anova_lm(model_supp, typ=2)
p_value_anova = anova_table_supp.loc['C(supp)', 'PR(>F)']

print(f"ANOVA F-test p-value for supplement variable: {p_value_anova}")


ANOVA F-test p-value for supplement variable: 0.060393371224128745


## Run multiple t-tests

While the 2-category ANOVA test is identical to a 2-tailed t-test, performing multiple t-tests leads to the multiple comparisons problem. To investigate this, look at the various sample groups you could create from the 2 features: 

In [9]:
for group in df.groupby(['supp', 'dose'])['len']:
    group_name = group[0]
    data = group[1]
    print(group_name)

('OJ', 0.5)
('OJ', 1.0)
('OJ', 2.0)
('VC', 0.5)
('VC', 1.0)
('VC', 2.0)


While bad practice, examine the effects of calculating multiple t-tests with the various combinations of these. To do this, generate all combinations of the above groups. For each pairwise combination, calculate the p-value of a 2-sided t-test. Print the group combinations and their associated p-value for the two-sided t-test.

In [11]:
# Your code here; reuse your t-test code above to calculate the p-value for a 2-sided t-test
# for all combinations of the supplement-dose groups listed above. 
# (Since there isn't a control group, compare each group to every other group.)

from itertools import combinations
groups = df.groupby(['supp', 'dose'])['len']
for (group1, data1), (group2, data2) in combinations(groups, 2):
    t_statistic, p_value = ttest_ind(data1, data2)
    print(f"t-test between {group1} and {group2}: p-value = {p_value}")

t-test between ('OJ', 0.5) and ('OJ', 1.0): p-value = 8.357559281443774e-05
t-test between ('OJ', 0.5) and ('OJ', 2.0): p-value = 3.4018585295016214e-07
t-test between ('OJ', 0.5) and ('VC', 0.5): p-value = 0.005303661339923052
t-test between ('OJ', 0.5) and ('VC', 1.0): p-value = 0.04223992429368205
t-test between ('OJ', 0.5) and ('VC', 2.0): p-value = 7.025409196997986e-06
t-test between ('OJ', 1.0) and ('OJ', 2.0): p-value = 0.03736279585664383
t-test between ('OJ', 1.0) and ('VC', 0.5): p-value = 1.3372624230559434e-08
t-test between ('OJ', 1.0) and ('VC', 1.0): p-value = 0.0007807261651774468
t-test between ('OJ', 1.0) and ('VC', 2.0): p-value = 0.09583711277517494
t-test between ('OJ', 2.0) and ('VC', 0.5): p-value = 1.3381068810881244e-11
t-test between ('OJ', 2.0) and ('VC', 1.0): p-value = 2.3131084633597503e-07
t-test between ('OJ', 2.0) and ('VC', 2.0): p-value = 0.9637097790041267
t-test between ('VC', 0.5) and ('VC', 1.0): p-value = 6.492264598157612e-07
t-test between ('V

## Summary

In this lesson, you implemented the ANOVA technique to generalize testing methods to multiple groups and factors.